<a href="https://colab.research.google.com/github/soumik12345/AODNet/blob/master/notebooks/AODnet_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-944253be-b0ee-7df4-1549-20cd3e737097)


In [2]:
!git clone https://github.com/soumik12345/AODNet
%cd AODNet

Cloning into 'AODNet'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 96 (delta 50), reused 72 (delta 26), pack-reused 0
Unpacking objects: 100% (96/96), done.
/content/AODNet


In [3]:
!pip install -qq wandb

     |████████████████████████████████| 2.0MB 7.6MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 163kB 37.2MB/s 
     |████████████████████████████████| 133kB 33.8MB/s 
     |████████████████████████████████| 71kB 12.0MB/s 


In [4]:
import tensorflow as tf
from aodnet.training import Trainer
from aodnet.utils import download_dataset, init_wandb

In [5]:
download_dataset()

Downloading...
From: https://drive.google.com/uc?id=1sInD9Ydq8-x7WwqehE0EyRknMdSFPOat
To: /content/AODNet/Dehaze-NYU.zip
956MB [00:06, 153MB/s]


Unpacking Dataset
Done!!!


In [6]:
init_wandb(
    project_name='aodnet', experiment_name='aodnet_train_nyu',
    wandb_api_key='cf0947ccde62903d4df0742a58b8a54ca4c11673'
)

wandb: Currently logged in as: 19soumik-rakshit96 (use `wandb login --relogin` to force relogin)


In [7]:
trainer = Trainer()
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

with strategy.scope():
    trainer.build_datasets(
        dataset_path='./Dehazing', image_crop_size=256, buffer_size=1024,
        batch_size=16, val_split=0.1, plot_samples=False
    )
    trainer.build_model(build_shape=(1, 256, 256, 3))

Model: "aod_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  12        
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  12        
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  453       
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  885       
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  327       
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
Total params: 1,689
Trainable params: 1,689
Non-trainable params: 0
_________________________________________________________

In [8]:
trainer.train(epochs=10)

Epoch 1/10
1534/1534 [==============================] - ETA: 0s - loss: 0.1752 - peak_signal_noise_ratio: 60.7671

wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init


1534/1534 [==============================] - 205s 121ms/step - loss: 0.1751 - peak_signal_noise_ratio: 60.7696 - val_loss: 0.0188 - val_peak_signal_noise_ratio: 66.4284


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
1534/1534 [==============================] - 196s 121ms/step - loss: 0.0204 - peak_signal_noise_ratio: 66.2353 - val_loss: 0.0182 - val_peak_signal_noise_ratio: 66.5198
Epoch 3/10
1534/1534 [==============================] - 196s 121ms/step - loss: 0.0198 - peak_signal_noise_ratio: 66.3910 - val_loss: 0.0172 - val_peak_signal_noise_ratio: 66.8760
Epoch 4/10
1534/1534 [==============================] - 195s 120ms/step - loss: 0.0192 - peak_signal_noise_ratio: 66.6079 - val_loss: 0.0169 - val_peak_signal_noise_ratio: 67.0203
Epoch 5/10
1534/1534 [==============================] - 195s 120ms/step - loss: 0.0188 - peak_signal_noise_ratio: 66.7913 - val_loss: 0.0166 - val_peak_signal_noise_ratio: 67.1019
Epoch 6/10
1534/1534 [==============================] - 194s 120ms/step - loss: 0.0180 - peak_signal_noise_ratio: 67.0140 - val_loss: 0.0160 - val_peak_signal_noise_ratio: 67.3534
Epoch 7/10
1534/1534 [==============================] - 193s 119ms/step - loss: 0.0175 - peak_signal

In [17]:
trainer.save_weights('aodnet_weights/weights')

Saving model weights at ./checkpoints/aodnet_weights/weights...
Done!!!
